In [1]:
from systemtools.hayj import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from systemtools.logger import *
from annotator.annot import *
from datatools.jsonutils import *
from nlptools.tokenizer import *
from datatools.htmltools import *
from newssource.goodarticle.utils import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR, LinearSVC
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold

In [2]:
data = []
for file in sortedGlob("goodarticle*.json"):
    data += fromJsonFile(file)

In [3]:
stopwords = set(fileToStrList("stopwords.txt"))
startswithExcludes = set(fileToStrList("startswith-excludes.txt"))

In [4]:
newData = []
for i in range(len(data)):
    data[i]["text"] = newsPreclean(data[i]["text"], startswithExcludes=startswithExcludes)
    if len(data[i]["text"]) > 0:
        newData.append(data[i])
data = newData

In [5]:
bp(data, 2)
print(len(data))

[
  {
    authors: [ Daniel Marans, Reporter ],
    dataset: twinews,
    domain: huffingtonpost.com,
    relevance: 1.0,
    text: In a Saturday blogpost, Democratic National Committee chairman Tom Perez vowed to enact reforms to t,
    title: DNC Chairman Promises To Reform Party's Presidential Nominating Process | HuffPost,
    url: https://www.huffingtonpost.com/entry/tom-perez-dnc-chair-reform-presidential-nominating-process_us_5
  },
  {
    authors: [ Dirk Verdoorn ],
    dataset: twinews,
    domain: kcra.com,
    relevance: 0.0,
    text: Hearst Television participates in various affiliate marketing programs, which means we may get paid ,
    title: Sacramento and Northern California Weather Updates – KCRA 3 News,
    url: https://www.kcra.com/weather?platform=hootsuite
  },
  ...,
  {
    authors: None,
    dataset: newsandblog,
    domain: nypost.com,
    relevance: 0.6,
    text: MORE ON: Tiler Peck wants you to know there's more to being a ballerina than standing on your t

In [8]:
for i, current in enumerate(data):
    if current["relevance"] == 0.0:
        text = current["text"]
        text = newsPreclean(text, startswithExcludes=startswithExcludes)
        bp(basicFeatures(text, stopwords=stopwords, asDict=True), 5)
        print()
        print(text)
        print()
        print()
        print()
        if i >= 3:
            break

{ 'encodingProbCount': 0, 'hasUpperRatio': 0.1875, 'htmlCharCount': 0, 'htmlCharRatio': 0.0, 'length': 218, 'lineStartWithNonWordRatio': 0.5, 'linesCount': 2, 'longLinesCount': 1, 'longLinesRatio': 0.5, 'meanLinesLength': 108.5, 'nonSWPMeanOverlap': 1.1111111111111112, 'nonWordCount': 1, 'nonWordRatio': 0.03125, 'shortLinesCount': 0, 'shortLinesRatio': 0.0, 'stopwordsPunctRatio': 0.375, 'tokensCount': 32, 'tooLongDocument': False, 'upperWordCount': 1, 'upperWordRatio': 0.03125 }

Hearst Television participates in various affiliate marketing programs, which means we may get paid commissions on purchases made through our links to retailer sites.
©2018, Hearst Television Inc. on behalf of KCRA-TV.



{ 'encodingProbCount': 0, 'hasUpperRatio': 0.22727272727272727, 'htmlCharCount': 0, 'htmlCharRatio': 0.0, 'length': 277, 'lineStartWithNonWordRatio': 0.0, 'linesCount': 1, 'longLinesCount': 1, 'longLinesRatio': 1.0, 'meanLinesLength': 277.0, 'nonSWPMeanOverlap': 1.1904761904761905, 'nonWordCo

In [9]:
X = np.array([basicFeatures(current["text"], stopwords=stopwords) for current in data])

In [10]:
y = np.array([continuous2discret(current["relevance"], [0.51]) for current in data])

In [11]:
bp(X)
bp(y)
print(len(y))

[[0.00000000e+00 2.59600000e+03 1.00000000e+01 ... 1.63727960e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.18000000e+02 2.00000000e+00 ... 1.87500000e-01
  5.00000000e-01 0.00000000e+00]
 [0.00000000e+00 7.22300000e+03 2.90000000e+01 ... 1.85550082e-01
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 2.02300000e+03 1.00000000e+00 ... 2.37942122e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.23400000e+03 1.00000000e+00 ... 1.96864111e-01
  0.00000000e+00 7.00000000e+00]
 [0.00000000e+00 2.97100000e+03 1.00000000e+00 ... 1.69201521e-01
  0.00000000e+00 0.00000000e+00]]
[1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0
 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1
 1 0 1 1 0 0 1 1 1 0 0 1 

In [12]:
if False:
    from sklearn.metrics import make_scorer
    scorer = make_scorer(accuracy, greater_is_better=True) # scorer(clf, X, y)

In [13]:
param_grid = \
{
    'loss': ['squared_hinge'], # 'hinge'
    'penalty': ['l1', 'l2',], # l1, l2
    'C': list(range(0, 20 + 1)),
    'multi_class': ['ovr'], # , 'crammer_singer'
    'dual': [False, True],
    'random_state': [0],
}

In [14]:
clf = GridSearchCV(LinearSVC(), param_grid=param_grid, scoring='accuracy',
                   cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True), n_jobs=cpuCount(),
                  error_score=0.0)

In [15]:
clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             error_score=0.0,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=8,
             param_grid={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20],
                         'dual': [False, True], 'loss': ['squared_hinge'],
                         'multi_class': ['ovr'], 'penalty': ['l1', 'l2'],
                         'random_state': [0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [16]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print(clf.best_score_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

Best parameters set found on development set:

{'C': 6, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 0}
0.9236363636363636

Grid scores on development set:

0.000 (+/-0.000) for {'C': 0, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l1', 'random_state': 0}
0.000 (+/-0.000) for {'C': 0, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 0}
0.000 (+/-0.000) for {'C': 0, 'dual': True, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l1', 'random_state': 0}
0.000 (+/-0.000) for {'C': 0, 'dual': True, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 0}
0.899 (+/-0.041) for {'C': 1, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l1', 'random_state': 0}
0.916 (+/-0.054) for {'C': 1, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 0}
0.000 (+/-0.000) for {'C': 

In [ ]:
best = clf.best_estimator_

In [29]:
predictions = best.predict(X)

In [30]:
predictions

array([1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,

In [31]:
wellClassified = 0
for i in range(len(predictions)):
    if predictions[i] == y[i]:
        wellClassified += 1
    else:
        print("prediction: " + str(predictions[i]))
        print("true label: " + str(y[i]))
        print(data[i]["text"])
        print()
        print()
        print()
        print()
print(wellClassified / len(y))

prediction: 0
true label: 1
Email
A woman accused of driving drunk as she livestreamed a crash that killed her younger sister says she made the video to help pay for her sibling’s funeral expenses.
Jacqueline Sanchez Estrada is shown in a photo posted to a GoFundMe page for her family.
Obdulia Sanchez, who has been detained since her July 21 arrest, described her reasoning in a four-page, double-sided letter penned  to a reporter at KGPE-TV in Fresno.
“I made that video because I knew I had more than 5,000 followers,” the 18-year-old Stockton resident wrote. “It was the only way my sister would get a decent burial. I would never expose my sister like that. I anticipated the public donating money because my family isn't rich."
Sanchez apologized for making the video, saying, “I look awful, but I accomplished my goal.”




prediction: 0
true label: 1
David O'Reill, scientific director for British American Tobacco said nicotine makes a person's brain work better
Health experts have brande

In [17]:
bestParams = {'C': 6, 'dual': False, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 0}

In [18]:
clf = LinearSVC(**bestParams)

In [19]:
clf.fit(X, y)

LinearSVC(C=6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [20]:
best = clf

In [24]:
import pickle

In [27]:
serialize(clf, "best.pickle")

In [28]:
best = deserialize("best.pickle")

In [26]:
s

b'\x80\x03csklearn.svm.classes\nLinearSVC\nq\x00)\x81q\x01}q\x02(X\x04\x00\x00\x00dualq\x03\x89X\x03\x00\x00\x00tolq\x04G?\x1a6\xe2\xeb\x1cC-X\x01\x00\x00\x00Cq\x05K\x06X\x0b\x00\x00\x00multi_classq\x06X\x03\x00\x00\x00ovrq\x07X\r\x00\x00\x00fit_interceptq\x08\x88X\x11\x00\x00\x00intercept_scalingq\tK\x01X\x0c\x00\x00\x00class_weightq\nNX\x07\x00\x00\x00verboseq\x0bK\x00X\x0c\x00\x00\x00random_stateq\x0cK\x00X\x08\x00\x00\x00max_iterq\rM\xe8\x03X\x07\x00\x00\x00penaltyq\x0eX\x02\x00\x00\x00l2q\x0fX\x04\x00\x00\x00lossq\x10X\r\x00\x00\x00squared_hingeq\x11X\x08\x00\x00\x00classes_q\x12cnumpy.core.multiarray\n_reconstruct\nq\x13cnumpy\nndarray\nq\x14K\x00\x85q\x15C\x01bq\x16\x87q\x17Rq\x18(K\x01K\x02\x85q\x19cnumpy\ndtype\nq\x1aX\x02\x00\x00\x00i8q\x1bK\x00K\x01\x87q\x1cRq\x1d(K\x03X\x01\x00\x00\x00<q\x1eNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x1fb\x89C\x10\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00q tq!bX\x05\x00\x00\x00coef_q"h\x13h\x14K\x00\x85q#h\x16\x87q$Rq